## PySpark Random Sample
PySpark provides a pyspark.sql.DataFrame.sample(), pyspark.sql.DataFrame.sampleBy(), RDD.sample(), and RDD.takeSample() methods to get the random sampling subset from the large dataset.

PySpark sampling (pyspark.sql.DataFrame.sample()) is a mechanism to get random sample records from the dataset, this is helpful when you have a larger dataset and wanted to analyze/test a subset of the data for example 10% of the original file.

The syntax of the sample() function<br>
sample(withReplacement, fraction, seed=None)<br>
* fraction – Fraction of rows to generate, range [0.0, 1.0]. Note that it doesn’t guarantee to provide the exact number of the fraction of records.
* seed – Seed for sampling (default a random seed). Used to reproduce the same random sampling.
* withReplacement – Sample with replacement or not (default False).

### Using fraction to get a random sample in PySpark
By using fraction between 0 to 1, it returns the approximate number of the fraction of the dataset. For example, 0.1 returns 10% of the rows. However, this does not guarantee it returns the exact 10% of the records.

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

In [2]:
df=spark.range(100)
print(df.sample(0.06).collect())

[Row(id=26), Row(id=28), Row(id=41), Row(id=47), Row(id=49)]


### Using seed to reproduce the same Samples in PySpark
Every time you run a sample() function it returns a different set of sampling records, however sometimes during the development and testing phase you may need to regenerate the same sample every time as you need to compare the results from your previous run. To get consistent same random sampling uses the same slice value for every run. Change slice value to get different results.

In [3]:
print(df.sample(0.1,123).collect())

print(df.sample(0.1,123).collect())

print(df.sample(0.1,456).collect())

[Row(id=35), Row(id=38), Row(id=41), Row(id=45), Row(id=71), Row(id=84), Row(id=87), Row(id=99)]
[Row(id=35), Row(id=38), Row(id=41), Row(id=45), Row(id=71), Row(id=84), Row(id=87), Row(id=99)]
[Row(id=22), Row(id=33), Row(id=35), Row(id=41), Row(id=53), Row(id=80), Row(id=83), Row(id=87), Row(id=92)]


### Sample withReplacement (May contain duplicates)
Some times you may need to get a random sample with repeated values. By using the value true, results in repeated values.

In [4]:
print(df.sample(True,0.3,123).collect()) #with Duplicates

print(df.sample(0.3,123).collect()) #No duplicates

[Row(id=0), Row(id=5), Row(id=9), Row(id=11), Row(id=13), Row(id=16), Row(id=17), Row(id=26), Row(id=26), Row(id=37), Row(id=41), Row(id=45), Row(id=49), Row(id=50), Row(id=50), Row(id=57), Row(id=58), Row(id=58), Row(id=65), Row(id=66), Row(id=71), Row(id=74), Row(id=77), Row(id=80), Row(id=81), Row(id=82), Row(id=84), Row(id=88), Row(id=90), Row(id=91), Row(id=91), Row(id=92), Row(id=94), Row(id=96)]
[Row(id=0), Row(id=4), Row(id=12), Row(id=15), Row(id=19), Row(id=21), Row(id=23), Row(id=24), Row(id=25), Row(id=28), Row(id=29), Row(id=34), Row(id=35), Row(id=36), Row(id=38), Row(id=41), Row(id=45), Row(id=47), Row(id=50), Row(id=52), Row(id=59), Row(id=63), Row(id=65), Row(id=71), Row(id=82), Row(id=84), Row(id=87), Row(id=94), Row(id=99)]


### Stratified sampling in PySpark
You can get Stratified sampling in PySpark without replacement by using sampleBy() method. It returns a sampling fraction for each stratum. If a stratum is not specified, it takes zero as the default.

sampleBy() Syntax<br>
sampleBy(col, fractions, seed=None)
* col – column name from DataFrame
* fractions – It’s Dictionary type takes key and value.

In [5]:
df2=df.select((df.id % 3).alias("key"))
print(df2.sampleBy("key", {0: 0.1, 1: 0.2},0).collect())

[Row(key=0), Row(key=0), Row(key=1), Row(key=1), Row(key=0), Row(key=1), Row(key=0), Row(key=1), Row(key=0), Row(key=0), Row(key=1), Row(key=1), Row(key=0)]


### PySpark RDD Sample
PySpark RDD also provides sample() function to get a random sampling, it also has another signature takeSample() that returns an Array[T].

PySpark RDD sample() function returns the random sampling similar to DataFrame and takes a similar types of parameters but in a different order.

sample() of RDD returns a new RDD by selecting random sampling. Below is a syntax.<br>
sample(self, withReplacement, fraction, seed=None)

In [6]:
rdd = spark.sparkContext.range(0,100)
print(rdd.sample(False,0.1,0).collect())
print(rdd.sample(True,0.3,123).collect())

[23, 48, 53, 60, 72, 87, 91, 96, 98]
[0, 11, 16, 18, 19, 23, 23, 24, 26, 26, 27, 29, 35, 38, 47, 49, 54, 54, 55, 61, 61, 66, 68, 81, 81, 82, 85, 97, 99]


RDD takeSample() Syntax <br>
RDD takeSample() is an action hence you need to careful when you use this function as it returns the selected sample records to driver memory. Returning too much data results in an out-of-memory error similar to collect().

takeSample(self, withReplacement, num, seed=None) 

In [7]:
print(rdd.takeSample(False,10,0))
print(rdd.takeSample(True,30,123))

[18, 60, 51, 68, 22, 1, 35, 84, 75, 72]
[72, 91, 55, 86, 37, 49, 34, 46, 63, 21, 81, 17, 20, 84, 29, 46, 84, 14, 59, 7, 80, 25, 60, 59, 54, 22, 34, 83, 82, 25]
